## JSON 파일 저장(ver2)

In [3]:
import pymysql
from datetime import datetime
from dateutil import parser
import json
import os
from dotenv import load_dotenv

# 날짜 상수
START_DATE_STATIC = datetime(2000, 1, 1)
END_DATE_STATIC = datetime.today()

# .env 파일 로드
load_dotenv()

# DB 연결
def get_connection():
    host = os.getenv("DB_HOST")
    user = os.getenv("DB_USER")
    password = os.getenv("DB_PASSWORD")
    db = os.getenv("DB_NAME")
    port = int(os.getenv("DB_PORT", "3306"))

    if not all([host, user, password, db]):
        raise ValueError("필수 DB 환경 변수가 .env에 누락되어 있습니다.")

    conn = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db=db,
        port=port,
        charset='utf8',
        connect_timeout=60,
        read_timeout=300,
        write_timeout=300
    )
    print(f"\nDB 연결 성공: {db}\n")
    return conn, db

# 테이블 목록 조회
def get_table_list(cursor, db_name):
    cursor.execute(f"SHOW TABLES FROM `{db_name}`")
    return [row[0] for row in cursor.fetchall()]

# datetime 컬럼 조회
def get_datetime_column(cursor, db_name, table):
    cursor.execute("""
        SELECT COLUMN_NAME, DATA_TYPE
        FROM information_schema.columns
        WHERE table_schema=%s AND table_name=%s
    """, (db_name, table))
    for col, dtype in cursor.fetchall():
        if dtype.lower() in ('datetime','timestamp','date'):
            return col
    return None

# 평균 row 길이 조회
def get_avg_row_length(cursor, db_name, table):
    cursor.execute("""
        SELECT AVG_ROW_LENGTH
        FROM information_schema.tables
        WHERE table_schema=%s AND table_name=%s
    """, (db_name, table))
    result = cursor.fetchone()
    return result[0] if result and result[0] else 0

# 전체 row 수 조회
def get_row_count(cursor, db_name, table):
    cursor.execute(f"SELECT COUNT(*) FROM `{db_name}`.`{table}`")
    return cursor.fetchone()[0]

# 주/월/연도별 분포 조회 (1회 스캔)
def get_distribution(cursor, db_name, table, datetime_col, unit):
    if unit=='year':
        period_sql = "YEAR(`{col}`)"
        label_fmt = lambda x: str(x)
    elif unit=='month':
        period_sql = "DATE_FORMAT(`{col}`, '%Y-%m')"
        label_fmt = lambda x: x
    elif unit=='week':
        period_sql = "YEARWEEK(`{col}`,3)"
        label_fmt = lambda x: str(x)
    else:
        raise ValueError("단위 오류")
    col = datetime_col
    sql = f"""
        SELECT {period_sql.replace('{col}',col)} AS period, COUNT(*) AS cnt
        FROM `{db_name}`.`{table}`
        WHERE `{col}` NOT IN ('0000-00-00','0000-00-00 00:00:00')
          AND `{col}` BETWEEN '1900-01-01' AND '2100-01-01'
        GROUP BY period
    """
    cursor.execute(sql)
    dist = {}
    for period, cnt in cursor.fetchall():
        label = label_fmt(period)
        dist[label] = cnt
    return dist

# 안전한 날짜 포맷
def safe_date_format(dt):
    return dt.strftime('%Y-%m-%d') if hasattr(dt,'strftime') else str(dt)

# 메인 함수
def main():
    conn, db_name = get_connection()
    cursor = conn.cursor()
    tables = get_table_list(cursor, db_name)

    result = {
        "총 테이블 수": len(tables),
        "총 테이블 명 목록": tables,
        "날짜 컬럼이 있는 테이블 수": 0,
        "날짜 컬럼이 없는 테이블 수": 0,
        "데이터가 없는 테이블 수": 0,
        "평균 row length가 0인 테이블 수": 0,
        "건너뛴 테이블 수": 0,
        "건너뛴 테이블 목록": [],
        "평균 row length가 0인 테이블 목록": [],
        "날짜 컬럼이 있는 테이블 명 목록": [],
        "날짜 컬럼이 없는 테이블 명 목록": [],
        "데이터가 없는 테이블 목록": [],
        "날짜 컬럼이 있는 테이블": {},
        "날짜 컬럼이 없는 테이블": {},
        "데이터가 없는 테이블": {}
    }

    for table in tables:
        # 현재 분석 중인 테이블 출력
        print(f"분석 중 테이블: {table}")
        row_count = get_row_count(cursor, db_name, table)
        avg_len = get_avg_row_length(cursor, db_name, table)

        if row_count == 0:
            result["데이터가 없는 테이블 수"] += 1
            result["데이터가 없는 테이블 목록"].append(table)
            result["데이터가 없는 테이블"][table] = {"startDate": None, "endDate": None, "week": {}, "month": {}, "year": {}}
            continue

        if avg_len == 0:
            result["평균 row length가 0인 테이블 수"] += 1
            result["평균 row length가 0인 테이블 목록"].append(table)
            continue

        datetime_col = get_datetime_column(cursor, db_name, table)
        if not datetime_col:
            # 날짜 컬럼 없음 → 균등 분포 처리
            result["날짜 컬럼이 없는 테이블 수"] += 1
            result["날짜 컬럼이 없는 테이블 명 목록"].append(table)
            from dateutil.relativedelta import relativedelta
            from datetime import timedelta
            def ranges(start, end, unit):
                lst = []
                cur = start
                while cur < end:
                    if unit == 'week':
                        nxt = cur + timedelta(days=7)
                        lbl = f"{cur:%Y-W%U}"
                    elif unit == 'month':
                        nxt = cur + relativedelta(months=1)
                        lbl = f"{cur:%Y-%m}"
                    else:
                        nxt = cur + relativedelta(years=1)
                        lbl = f"{cur:%Y}"
                    lst.append((lbl, cur, nxt))
                    cur = nxt
                return lst
            table_res = {"startDate": safe_date_format(START_DATE_STATIC), "endDate": safe_date_format(END_DATE_STATIC)}
            for unit in ['week', 'month', 'year']:
                rngs = ranges(START_DATE_STATIC, END_DATE_STATIC, unit)
                total_mb = row_count * avg_len / 1024 / 1024
                per_mb = total_mb / len(rngs) if rngs else 0
                table_res[unit] = {lbl: f"{per_mb:,.3f}" for lbl, _, _ in rngs}
            result["날짜 컬럼이 없는 테이블"][table] = table_res
            continue

        # datetime 컬럼 존재
        result["날짜 컬럼이 있는 테이블 수"] += 1
        result["날짜 컬럼이 있는 테이블 명 목록"].append(table)

        # 전체 valid 분포 조회
        dist_year = get_distribution(cursor, db_name, table, datetime_col, 'year')
        dist_month = get_distribution(cursor, db_name, table, datetime_col, 'month')
        dist_week = get_distribution(cursor, db_name, table, datetime_col, 'week')
        valid_count = sum(dist_year.values())
        invalid_count = row_count - valid_count

        # 기간별 용량 계산
        table_res = {'startDate': None, 'endDate': None, 'week': {}, 'month': {}, 'year': {}}
        cursor.execute(f"SELECT MIN(`{datetime_col}`), MAX(`{datetime_col}`) FROM `{db_name}`.`{table}` WHERE `{datetime_col}` NOT IN ('0000-00-00','0000-00-00 00:00:00')")
        sd, ed = cursor.fetchone()
        table_res['startDate'] = safe_date_format(parser.parse(str(sd))) if sd else safe_date_format(START_DATE_STATIC)
        table_res['endDate'] = safe_date_format(parser.parse(str(ed))) if ed else safe_date_format(END_DATE_STATIC)
        for unit, dist in (('year', dist_year), ('month', dist_month), ('week', dist_week)):
            ures = {}
            for lbl, cnt in dist.items():
                ures[lbl] = f"{cnt * avg_len / 1024 / 1024:,.3f}"
            if invalid_count > 0:
                ures['기타'] = f"{invalid_count * avg_len / 1024 / 1024:,.3f}"
            table_res[unit] = ures
        result["날짜 컬럼이 있는 테이블"][table] = table_res

    # 결과 JSON 저장
    jsf = f"db_storage_summary_{datetime.now():%Y%m%d_%H%M%S}.json"
    with open(jsf, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    print(f"완료: {jsf}")

    cursor.close()
    conn.close()

if __name__ == '__main__':
    main()



DB 연결 성공: meetu_db

분석 중 테이블: account
분석 중 테이블: admin
분석 중 테이블: advertisement
분석 중 테이블: application
분석 중 테이블: bookmark
분석 중 테이블: calendarEvent
분석 중 테이블: chatMessage
분석 중 테이블: chatRoom
분석 중 테이블: communityComment
분석 중 테이블: communityLike
분석 중 테이블: communityPost
분석 중 테이블: communityTag
분석 중 테이블: company
분석 중 테이블: companyBlock
분석 중 테이블: companyFollow
분석 중 테이블: coverLetter
분석 중 테이블: coverLetterContent
분석 중 테이블: coverLetterContentFeedback
분석 중 테이블: coverLetterContentFitAnalysis
분석 중 테이블: customerSupport
분석 중 테이블: interviewReview
분석 중 테이블: jobCategory
분석 중 테이블: jobPosting
분석 중 테이블: jobPostingJobCategory
분석 중 테이블: jobPostingViewLog
분석 중 테이블: location
분석 중 테이블: notification
분석 중 테이블: offer
분석 중 테이블: payment
분석 중 테이블: profile
분석 중 테이블: resume
분석 중 테이블: resumeContent
분석 중 테이블: resumeViewLog
분석 중 테이블: systemLog
완료: db_storage_summary_20250625_232014.json


## JSON -> HTML(Jupyter Notebook)

In [11]:
import glob
import json
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

pio.renderers.default = "notebook_connected"  # 또는 "iframe_connected"

# ───────────────────────────────────────────────────────────────
# 1. JSON 파일 찾기
json_files = sorted(glob.glob("db_storage_summary_20250625_232014.json"), reverse=True)
if not json_files:
    raise FileNotFoundError("db_storage_summary_*.json 파일을 찾을 수 없습니다.")
json_path = json_files[0]
print(f"▶ 로드할 JSON: {json_path}")

# 2. JSON 불러오기
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 3. 전체 요약 출력
summary_keys = [
    "총 테이블 수",
    "날짜 컬럼이 있는 테이블 수",
    "날짜 컬럼이 없는 테이블 수",
    "데이터가 없는 테이블 수",
    "건너뛴 테이블 수",
    "평균 row length가 0인 테이블 수"
]

print("\n=== 전체 요약 ===")
for key in summary_keys:
    print(f"{key}: {data.get(key, 0)}")

# ───────────────────────────────────────────────────────────────
# 4. 날짜 컬럼이 있는 테이블 – 연도별 라인차트
tables_with_date = data.get("날짜 컬럼이 있는 테이블", {})
fig1 = go.Figure()
table_names_1, totals_1 = [], []

for tbl, content in tables_with_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = sorted(year_data.keys())
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig1.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_1.append(tbl)
    totals_1.append(f"{sum(y):,.2f}")

fig1.update_layout(
    title="날짜 컬럼이 있는 테이블 – 연도별 용량 추이",
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig1, full_html=False, include_plotlyjs='cdn')))

# ───────────────────────────────────────────────────────────────
# 5. 날짜 컬럼이 있는 테이블 – 총합 용량 요약표 (가로 배치)
if table_names_1:
    fig_table1 = go.Figure(data=[go.Table(
        header=dict(
            values=table_names_1,
            fill_color="lightblue",
            align="center"
        ),
        cells=dict(
            # one list per column, each list holds the single value for that column
            values=[[v] for v in totals_1],
            align="center"
        )
    )])
    fig_table1.update_layout(title="날짜 컬럼이 있는 테이블 – 총합 용량 (가로)")
    display(HTML(pio.to_html(fig_table1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 6. 날짜 컬럼이 없는 테이블 – 연도별 라인차트
tables_without_date = data.get("날짜 컬럼이 없는 테이블", {})
fig2 = go.Figure()
table_names_2, totals_2 = [], []

for tbl, content in tables_without_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = sorted(year_data.keys())
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig2.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_2.append(tbl)
    totals_2.append(f"{sum(y):,.2f}")

fig2.update_layout(
    title="날짜 컬럼이 없는 테이블 – 연도별 용량 추이",
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 7. 날짜 컬럼이 없는 테이블 – 총합 용량 요약표 (가로 배치)
if table_names_2:
    fig_table2 = go.Figure(data=[go.Table(
        header=dict(
            values=table_names_2,
            fill_color="lightgray",
            align="center"
        ),
        cells=dict(
            values=[[v] for v in totals_2],
            align="center"
        )
    )])
    fig_table2.update_layout(title="날짜 컬럼이 없는 테이블 – 총합 용량 (가로)")
    display(HTML(pio.to_html(fig_table2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 8. 데이터가 없는 테이블: 단순 리스트 출력
empty_tables = data.get("데이터가 없는 테이블 목록", [])
print("\n=== 데이터가 없는 테이블 목록 ===")
if empty_tables:
    for tbl in empty_tables:
        print(f"- {tbl}")
else:
    print("없음")


▶ 로드할 JSON: db_storage_summary_20250625_232014.json

=== 전체 요약 ===
총 테이블 수: 34
날짜 컬럼이 있는 테이블 수: 34
날짜 컬럼이 없는 테이블 수: 0
데이터가 없는 테이블 수: 0
건너뛴 테이블 수: 0
평균 row length가 0인 테이블 수: 0



=== 데이터가 없는 테이블 목록 ===
없음


In [13]:
import glob
import json
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from IPython.display import display, HTML
from datetime import datetime

pio.renderers.default = "notebook_connected"  # 또는 "iframe_connected"

# ───────────────────────────────────────────────────────────────
# 1. JSON 파일 찾기
json_files = sorted(glob.glob("db_storage_summary_20250625_232014.json"), reverse=True)
if not json_files:
    raise FileNotFoundError("db_storage_summary_*.json 파일을 찾을 수 없습니다.")
json_path = json_files[0]
print(f"▶ 로드할 JSON: {json_path}")

# 2. JSON 불러오기
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 3. 전체 요약 출력
summary_keys = [
    "총 테이블 수",
    "날짜 컬럼이 있는 테이블 수",
    "날짜 컬럼이 없는 테이블 수",
    "데이터가 없는 테이블 수",
    "건너뛴 테이블 수",
    "평균 row length가 0인 테이블 수"
]

print("\n=== 전체 요약 ===")
for key in summary_keys:
    print(f"{key}: {data.get(key, 0)}")

# ───────────────────────────────────────────────────────────────
# 4. 날짜 컬럼이 있는 테이블 – 연도별 라인차트
tables_with_date = data.get("날짜 컬럼이 있는 테이블", {})
fig1 = go.Figure()
table_names_1, totals_1 = [], []

for tbl, content in tables_with_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = sorted(year_data.keys())
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig1.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_1.append(tbl)
    totals_1.append(f"{sum(y):,.2f}")

fig1.update_layout(
    title="날짜 컬럼이 있는 테이블 – 연도별 용량 추이",
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig1, full_html=False, include_plotlyjs='cdn')))

# ───────────────────────────────────────────────────────────────
# 5. 날짜 컬럼이 있는 테이블 – 총합 용량 요약표 (가로 배치)
if table_names_1:
    # uniform column width for readability
    col_widths_1 = [1000] * len(table_names_1)
    fig_table1 = go.Figure(data=[go.Table(
        columnwidth=col_widths_1,
        header=dict(
            values=table_names_1,
            fill_color="lightblue",
            align="center"
        ),
        cells=dict(
            values=[[v] for v in totals_1],
            align="center"
        )
    )])
    fig_table1.update_layout(margin=dict(t=50, b=20), title="날짜 컬럼이 있는 테이블 – 총합 용량 (가로)")
    display(HTML(pio.to_html(fig_table1, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 6. 날짜 컬럼이 없는 테이블 – 연도별 라인차트
tables_without_date = data.get("날짜 컬럼이 없는 테이블", {})
fig2 = go.Figure()
table_names_2, totals_2 = [], []

for tbl, content in tables_without_date.items():
    year_data = content.get("year", {})
    if not year_data:
        continue
    x = sorted(year_data.keys())
    y = [float(year_data[y].replace(",", "")) for y in x]

    fig2.add_trace(go.Scatter(x=x, y=y, mode="lines+markers", name=tbl))
    table_names_2.append(tbl)
    totals_2.append(f"{sum(y):,.2f}")

fig2.update_layout(
    title="날짜 컬럼이 없는 테이블 – 연도별 용량 추이",
    xaxis_title="연도",
    yaxis_title="용량 (MB)",
    template="plotly_white"
)
display(HTML(pio.to_html(fig2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 7. 날짜 컬럼이 없는 테이블 – 총합 용량 요약표 (가로 배치)
if table_names_2:
    col_widths_2 = [200] * len(table_names_2)
    fig_table2 = go.Figure(data=[go.Table(
        columnwidth=col_widths_2,
        header=dict(
            values=table_names_2,
            fill_color="lightgray",
            align="center"
        ),
        cells=dict(
            values=[[v] for v in totals_2],
            align="center"
        )
    )])
    fig_table2.update_layout(margin=dict(t=50, b=20), title="날짜 컬럼이 없는 테이블 – 총합 용량 (가로)")
    display(HTML(pio.to_html(fig_table2, full_html=False, include_plotlyjs=False)))

# ───────────────────────────────────────────────────────────────
# 8. 데이터가 없는 테이블: 단순 리스트 출력
empty_tables = data.get("데이터가 없는 테이블 목록", [])
print("\n=== 데이터가 없는 테이블 목록 ===")
if empty_tables:
    for tbl in empty_tables:
        print(f"- {tbl}")
else:
    print("없음")


▶ 로드할 JSON: db_storage_summary_20250625_232014.json

=== 전체 요약 ===
총 테이블 수: 34
날짜 컬럼이 있는 테이블 수: 34
날짜 컬럼이 없는 테이블 수: 0
데이터가 없는 테이블 수: 0
건너뛴 테이블 수: 0
평균 row length가 0인 테이블 수: 0



=== 데이터가 없는 테이블 목록 ===
없음
